# Prismatic structures of Li2MnO3

In this notebook I've developed a small interactive tool to view the final geometries of the prismatic models of Li$_2$MnO$_3$. Once the binder has loaded, just execute the cell below and then use the drop down menu's to choose the `model`, `Li concentration` and DFT `functional`.

<em> In order to execute the cell: select it with the left mouse button and press `Shift+Enter`.</em>

In [6]:
# Execute this cell to visualize the structures and compare them to the original model.

import os
import json
import imolecule

from pymatgen import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from monty.io import zopen
from ipywidgets import interact, fixed, Dropdown

# Lists of various structural models, Li concentrations and functional settings
models = ["Li2MnO3_type", "inverted_p2_like", "202639_Li2MnO3_pristine",
"Li2MnO3_type_O_shifted", "inverted_p2_like_O_shifted", "symm_inverted_p2_like"]
li_concentrations = ["0", "0.25"]
functional_list = ["scan" , "pbeu_Mn3.9", "hse"]

# Set the atom sizes and colors for imolecule
element_properties={
    "Li": { "color": 0x66ff66, "radius": 0.85 },
    "O": { "color": 0xff0000, "radius": 0.7 },
    "Mn": { "color": 0xf00fff, "radius": 1.35 },
}

data_dir = os.path.join(os.getcwd(), "li2mno3_prism")

def get_energy(data_file):
    """
    Quick method for extracting the final energy from a calculation.
    """

    with zopen(data_file) as file:
        data = json.load(file)
        return data["output"]["final_energy"]

def visualize_prism(data_dir, model, li_conc, functional):
    
    conc_dir = str(li_conc) + "_li"
    
    if li_conc == "0":
        orig_path = os.path.join(data_dir, conc_dir, model, model + "_charged.json")
    else:
        orig_path = os.path.join(data_dir, conc_dir, model, model + ".json")
        
    orig_cat = Structure.from_file(orig_path)
    super_orig = orig_cat.copy()
    super_orig.make_supercell([2, 2, 2])
    super_orig.to("cif", "temp.cif")
    print()
    print("Original Structure")
    print("------------------")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)
    
    final_path = os.path.join(data_dir, conc_dir, model, functional + "_relax", "final_cathode.cif")
    final_cat = Structure.from_file(final_path)
    if functional == "hse":
        spg = SpacegroupAnalyzer(final_cat)
        super_final = spg.get_conventional_standard_structure()
    else:
        super_final = final_cat.copy()
    super_final.make_supercell([2, 2, 2])
    super_final.to("cif", "temp.cif")
    print()
    print("Final Structure")
    print("---------------")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)
    
# Interactive function for visualizing the structures
interact(visualize_prism,
         data_dir=fixed(data_dir),
         model=models, 
         li_conc=Dropdown(options=li_concentrations,
                         description="Li concentration"),
         functional=functional_list);

interactive(children=(Dropdown(description='model', options=('Li2MnO3_type', 'inverted_p2_like', '202639_Li2Mn…

Note that the structures above can be rotated by clicking and dragging with the left mouse button. In case no structures are visible right after executing the cell, try changing the selection in one of the dropdown menu's.